In [1]:
model_lists = ["STGAN", "SIDPAMIwISTGAN", "SIDSTGAN"]
checkpoint_lists = ['latest'] #[10*i for i in range(0,90)]

In [2]:
import cv2
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import math
from tqdm import tqdm
import os
from PIL import Image

In [3]:
def calculate_psnr(img1_tensor, img2_tensor):
    """Calculate PSNR score
    
    Parameters:
        img1_tensor (int) -- a image to compare, range [0, 255]
        img2_tensor (int) -- a image to compare, range [0, 255]
    """    
    img1 = ((img1_tensor + 1.0)*255.0/2.0).astype(np.float64)
    img2 = ((img2_tensor + 1.0)*255.0/2.0).astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

def calculate_ssim(img1_tensor, img2_tensor):
    """Calculate SSIM score
    
    Parameters:
        img1_tensor (int) -- a image to compare, range [0, 255]
        img2_tensor (int) -- a image to compare, range [0, 255]
    """
    def ssim(img1, img2):
        C1 = (0.01 * 255)**2
        C2 = (0.03 * 255)**2

        kernel = cv2.getGaussianKernel(11, 1.5)
        window = np.outer(kernel, kernel.transpose())

        mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
        mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
        mu1_sq = mu1**2
        mu2_sq = mu2**2
        mu1_mu2 = mu1 * mu2
        sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
        sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
        sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

        ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                                (sigma1_sq + sigma2_sq + C2))
        return ssim_map.mean()
    
    img1 = ((img1_tensor + 1.0)*255.0/2.0).astype(np.float64)
    img2 = ((img2_tensor + 1.0)*255.0/2.0).astype(np.float64)
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')
        
def calculate_dice(mask_gt, mask_pred):
    mask_gt = (mask_gt + 1.0)/2.0
    mask_pred = (mask_pred + 1.0)/2.0
    
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    
    volume_intersect = mask_gt*mask_pred
    volume_intersect = volume_intersect.sum()
    
    return 2*volume_intersect / volume_sum 

In [4]:
def compute_score(groudtruth_path, predicted_path, color_image = True):
    def get_score(func, score_dict, index, img_groudtruth, img_predicted):
        score_dict['total'] += func(img_groudtruth, img_predicted)
        if index%10 < 5:
            score_dict['indoors'] += func(img_groudtruth, img_predicted)            
        else:
            score_dict['outdoors'] += func(img_groudtruth, img_predicted) 
            
    score = list()
    if color_image:
        score.append({'name': 'PNSR', 'func': calculate_psnr, 'total': 0, 'indoors': 0, 'outdoors': 0})
    else:
        score.append({'name': 'Dice', 'func': calculate_dice, 'total': 0, 'indoors': 0, 'outdoors': 0})
    score.append({'name': 'SSIM', 'func': calculate_ssim, 'total': 0, 'indoors': 0, 'outdoors': 0})
    
    groudtruth = list(paths.list_images(groudtruth_path))
    predicted = list(paths.list_images(predicted_path))

    for i in tqdm(range(len(predicted))):
        img_predicted = cv2.imread(predicted[i], color_image)
        img_groudtruth = cv2.imread(groudtruth[i], color_image)
        
        if color_image:
            img_predicted = cv2.cvtColor(img_predicted, cv2.COLOR_BGR2RGB)
            img_groudtruth = cv2.cvtColor(img_groudtruth, cv2.COLOR_BGR2RGB)
        
        img_predicted = (img_predicted/255.0-0.5)*2.0
        img_groudtruth = (img_groudtruth/255.0-0.5)*2.0

        get_score(score[0]['func'], score[0], i, img_groudtruth, img_predicted)
        get_score(score[1]['func'], score[1], i, img_groudtruth, img_predicted)

    print("___{}-{}_______________________:".format(groudtruth_path.split("\\")[-1],predicted_path.split("\\")[-1]))
    print("{}_score:".format(score[0]["name"]), score[0]["total"]/len(predicted), \
          ", {}_score:".format(score[1]["name"]), score[1]["total"]/len(predicted))
    print("{}_score_indoors:".format(score[0]["name"]), score[0]["indoors"]/len(predicted)*2, \
          ", {}_score:".format(score[1]["name"]), score[1]["indoors"]/len(predicted)*2)
    print("{}_score_outdoors:".format(score[0]["name"]), score[0]["outdoors"]/len(predicted)*2, \
          ", {}_score:".format(score[1]["name"]), score[1]["outdoors"]/len(predicted)*2)
    
    return score[0]["total"]/len(predicted), score[1]["total"]/len(predicted)

In [5]:
def plot_figure(ax, data, name):
    ax.plot(data, color='magenta', marker='o',mfc='pink' ) #plot the data
    ax.set_xticks(range(0,len(data)+1, 1))
    ax.set_ylabel('data')
    ax.set_xlabel('epoch')
    ax.set_title(name)

In [6]:
print("_____Experiment on Palm ROI images_____")
path = os.path.join(os. getcwd(), "RSSD\\_result_set")
groudtruth_path = os.path.join(path, "groudtruth__roi")     
path_dict = {"original": [os.path.join(path, "original__roi")]}
for name_list in model_lists:
    path_dict[name_list] = list()
    path_dict[f"{name_list}_HandSeg"] = list()

for index in checkpoint_lists:
    for name_list in model_lists:
        path_dict[name_list].append(os.path.join(path, f"shadowfree_{name_list}_rawsynthetic_{index}__roi"))      
        path_dict[f"{name_list}_HandSeg"].append(os.path.join(path, f"shadowfree_{name_list}_rawsynthetic_HandSeg_{index}__roi"))

draw_data = dict()
for model_name in list(path_dict.keys()):
    PNSR_list, SSIM_list = list(), list()
    
    for predicted_path in path_dict[model_name]: 
        PNSR, SSIM = compute_score(groudtruth_path, predicted_path, color_image = 1)
        PNSR_list.append(PNSR)
        SSIM_list.append(SSIM)
    
    if len(PNSR_list) > 1:
        draw_data[model_name] = [PNSR_list, SSIM_list]

_____Experiment on Palm ROI images_____


100%|██████████████████████████████████████████████████████████████████████████████| 1600/1600 [01:54<00:00, 13.94it/s]


___groudtruth__roi-original__roi_______________________:
PNSR_score: 15.949928830147785 , SSIM_score: 0.7601168050105723
PNSR_score_indoors: 16.416767732324637 , SSIM_score: 0.7848511873054395
PNSR_score_outdoors: 15.483089927970966 , SSIM_score: 0.7353824227157044


100%|██████████████████████████████████████████████████████████████████████████████| 1600/1600 [02:12<00:00, 12.04it/s]


___groudtruth__roi-shadowfree_STGAN_rawsynthetic_190__roi_______________________:
PNSR_score: 19.440413819421714 , SSIM_score: 0.7840667198241066
PNSR_score_indoors: 20.277426437769204 , SSIM_score: 0.8136439319638029
PNSR_score_outdoors: 18.603401201074185 , SSIM_score: 0.7544895076844046


...

___groudtruth__roi-shadowfree_SIDSTGAN_rawsynthetic_HandSeg_190__roi_______________________:
PNSR_score: 19.9327355578293 , SSIM_score: 0.7977170294761884
PNSR_score_indoors: 20.710126352369688 , SSIM_score: 0.8273520104469883
PNSR_score_outdoors: 19.155344763288937 , SSIM_score: 0.7680820485053879


100%|██████████████████████████████████████████████████████████████████████████████| 1600/1600 [01:56<00:00, 13.74it/s]


___groudtruth__roi-shadowfree_SIDPAMIwISTGAN_rawsynthetic_190__roi_______________________:
PNSR_score: 19.56079945932398 , SSIM_score: 0.7901856452580418
PNSR_score_indoors: 20.635715162994767 , SSIM_score: 0.8224213403239169
PNSR_score_outdoors: 18.48588375565312 , SSIM_score: 0.7579499501921637


100%|██████████████████████████████████████████████████████████████████████████████| 1600/1600 [01:54<00:00, 13.93it/s]

___groudtruth__roi-shadowfree_SIDPAMIwISTGAN_rawsynthetic_HandSeg_190__roi_______________________:
PNSR_score: 19.885948569203535 , SSIM_score: 0.7952757070601217
PNSR_score_indoors: 20.73754431383587 , SSIM_score: 0.8255233284412371
PNSR_score_outdoors: 19.034352824571187 , SSIM_score: 0.7650280856790035


In [13]:
# Visualize data
print("_____Experiment on Palm ROI images_____")
model_list = list(draw_data.keys())
if len(model_list) > 0:
    fig, axs = plt.subplots(nrows=len(model_list),ncols=2, figsize=(20,10))
    for model_id in range(len(model_list)): 
        plot_figure(axs[model_id, 0], draw_data[model_list[model_id]][0], f"PNSR score_{model_list[model_id]}")
        plot_figure(axs[model_id, 1], draw_data[model_list[model_id]][1], f"SSIM score_{model_list[model_id]}")
    plt.tight_layout()
    plt.show()

_____Experiment on Palm ROI images_____


In [14]:
print("_____Experiment on Palm Vein images_____")
path = os.path.join(os. getcwd(), "RSSD\\_result_set")
groudtruth_path = os.path.join(path, "groudtruth__vein")     
path_dict = {"original": [os.path.join(path, "original__vein")]}
for name_list in model_lists:
    path_dict[name_list] = list()
    path_dict[f"{name_list}_HandSeg"] = list()

for index in checkpoint_lists:
    for name_list in model_lists:
        path_dict[name_list].append(os.path.join(path, f"shadowfree_{name_list}_rawsynthetic_{index}__vein"))      
        path_dict[f"{name_list}_HandSeg"].append(os.path.join(path, f"shadowfree_{name_list}_rawsynthetic_HandSeg_{index}__vein"))

draw_data = dict()
for model_name in list(path_dict.keys()):
    Dice_list, SSIM_list = list(), list()
    
    for predicted_path in path_dict[model_name]: 
        Dice, SSIM = compute_score(groudtruth_path, predicted_path, color_image = 0)
        Dice_list.append(Dice)
        SSIM_list.append(SSIM)
    
    if len(PNSR_list) > 1:
        draw_data[model_name] = [Dice_list, SSIM_list]

_____Experiment on Palm Vein images_____


100%|█████████████████████████████████████████████████████████████████████████████| 1600/1600 [00:13<00:00, 121.77it/s]


___groudtruth__vein-original__vein_______________________:
Dice_score: 0.18091166002510659 , SSIM_score: 0.28561952839265703
Dice_score_indoors: 0.18255101131302184 , SSIM_score: 0.29443407137751015
Dice_score_outdoors: 0.17927230873719086 , SSIM_score: 0.27680498540780485


100%|██████████████████████████████████████████████████████████████████████████████| 1600/1600 [00:23<00:00, 68.36it/s]


___groudtruth__vein-shadowfree_STGAN_rawsynthetic_190__vein_______________________:
Dice_score: 0.1790118220105639 , SSIM_score: 0.2737803672787573
Dice_score_indoors: 0.18015249532470856 , SSIM_score: 0.27973359138754006
Dice_score_outdoors: 0.17787114869641873 , SSIM_score: 0.2678271431699753


...

___groudtruth__vein-shadowfree_SIDSTGAN_rawsynthetic_HandSeg_190__vein_______________________:
Dice_score: 0.1866645270795541 , SSIM_score: 0.2914003165215307
Dice_score_indoors: 0.18827938985708637 , SSIM_score: 0.29959001128591
Dice_score_outdoors: 0.18504966430202127 , SSIM_score: 0.283210621757151


100%|██████████████████████████████████████████████████████████████████████████████| 1600/1600 [00:23<00:00, 69.11it/s]


___groudtruth__vein-shadowfree_SIDPAMIwISTGAN_rawsynthetic_190__vein_______________________:
Dice_score: 0.18146813187586194 , SSIM_score: 0.2817311830597315
Dice_score_indoors: 0.18293645043910836 , SSIM_score: 0.29076076371443305
Dice_score_outdoors: 0.1799998133126161 , SSIM_score: 0.27270160240502916


100%|██████████████████████████████████████████████████████████████████████████████| 1600/1600 [00:22<00:00, 70.23it/s]

___groudtruth__vein-shadowfree_SIDPAMIwISTGAN_rawsynthetic_HandSeg_190__vein_______________________:
Dice_score: 0.1818963673629675 , SSIM_score: 0.2826223437494009
Dice_score_indoors: 0.18289290000172326 , SSIM_score: 0.2889883573641368
Dice_score_outdoors: 0.1808998347242117 , SSIM_score: 0.2762563301346657


In [15]:
# Visualize data
print("_____Experiment on Palm Vein images_____")
model_list = list(draw_data.keys())
if len(model_list) > 0:
    fig, axs = plt.subplots(nrows=len(model_list),ncols=2, figsize=(20,10))
    for model_id in range(len(model_list)): 
        plot_figure(axs[model_id, 0], draw_data[model_list[model_id]][0], f"Dice score_{model_list[model_id]}")
        plot_figure(axs[model_id, 1], draw_data[model_list[model_id]][1], f"SSIM score_{model_list[model_id]}")
    plt.tight_layout()
    plt.show()

_____Experiment on Palm Vein images_____
